In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import matplotlib.dates as mdates
import scipy
import numpy as np
from pylab import *
from scipy.optimize import curve_fit
from scipy.stats import linregress

In [3]:
df=pd.read_excel('sn_diana.xlsx')

In [4]:
df.tail()

,SN,m15B,Em15B,B,EB,V,Ev,i,Ei,Rv,E(B-V)
19,2007bd,1.234,0.012,-19.19,0.0440,-19.25,0.0320,-18.85,0.044,0.1218,0.058
20,2007le,0.971,0.016,-18.78,0.0319,-19.24,0.0300,-19.54,0.057,0.6596,0.388
21,2007on,1.860,0.020,-18.60,0.0077,-18.69,0.0066,-18.30,0.021,0.0245,0.007
22,2008bc,0.823,0.019,-19.56,0.0073,-19.51,0.0130,-18.78,0.054,0.0589,0.019
23,2008gp,0.970,0.100,-19.34,0.0530,-19.47,0.0520,-18.75,0.192,0.0686,0.098


In [95]:
x=df.m15B
y=df.B

In [96]:
#Método de York
#Y=B+Ax+e
B=linregress(x,y).intercept
A=linregress(x,y).slope

In [97]:
ox=df.Em15B
oy=df.EB

In [98]:
Wi=pd.DataFrame([])

In [99]:
#correlación entre los errores de x, y
r=[]
for i in range(0,len(df)):
    g=(ox[i]-mean(ox))*(oy[i]-mean(oy))
    h=sqrt((ox[i]-mean(ox))**2)
    k=sqrt((oy[i]-mean(oy))**2)
    r.append(g/(h*k))

In [100]:
w=[]
a= sqrt(ox*oy)
for i in range(0,len(df)):
    if i==14:
        k=0
    else:
        j=(ox[i] * A**2 * oy[i] - 2*A * a[i] * r[i] )
        c = ox[i]*oy[i]#/(ox[i] * A**2 * oy[i] - 2*A * a[i] * r )
        k= c/j
    w.append(k)

In [101]:
#Parámetros de la linea ajustada al error
#y=Bo+Aox
Bo=0
Ao=0

U=np.std(x)
V=np.std(y)

In [102]:
s=[]
j=0
k=0

for i in range(0, len(df)):
    if i==14:
        b=0
        j=( w[i] * b * V ) + j
        k=( w[i] * b * U) + k
    else:
        b=w[i]*( (U/oy[i]) + (B*ox[i]/V) - (B*U+V)*(r[i]/a[i]) )
        j=( w[i] * b * V ) + j
        k=( w[i] * b * U) + k

Ao=j/k
Ao

0.8928238433440517

In [103]:
Bo=mean(y)-Ao*mean(x)
Bo

-20.14454386231049

In [104]:
#Calcular el error
oA=0
j=0
for i in range(0, len(df)):
    j=w[i]*U**2
    
    
oA=sqrt(abs(1/j))
oA

14.903994311695776

In [105]:
oB=0
j=0
for i in range(0, len(df)):
    j=w[i]
oB=sqrt(j+mean(x)*oA**2)
oB

16.224402476520293

In [106]:
#calculo de S
S=0
#z=mean(a)
for i in range(0, len(df)):
    S= w[i]*( y[i]- x[i]*Ao-Bo)**2 + S  
S=abs(S)

In [107]:
eBo=oB*sqrt(S/(len(df)-2))

eAo=oA*sqrt(S/(len(df)-2))

In [108]:
eAo

0.3703709999530149

In [109]:
eBo

0.40318374008996005

In [110]:
#R de Pearson
R=0
for i in range(0, len(df)):
    R=(A+B*x[i]-y[i])**2 / ((.147*Ao)**2 + (.1788)**2) + R

In [111]:
T=0
for i in range(0, len(df)):
    T=(y[i]-mean(y))**2+T
sct=T

In [112]:
1-(T/R)

0.9999368077191888

In [113]:
j=h=k=l=z=0
for i in range(0, len(df)):
    j=x[i] * y[i] +j
    h= x[i]+h
    k=y[i]+k
    l=( x[i])**2 +l
    z= x[i]+z
f=h * k 
#b1=(j-f/len(df))/(l-z**2 /len(df))
b1=A

In [114]:
#bo=mean(y)-b1*mean(x)
bo=B

In [115]:
Sc=T/(len(df)-2)
Sc

0.05956193181818174

In [116]:
tc=(bo-Bo)/sqrt( Sc*(1/len(df) + mean(x)**2)/S )
tc

0.15408461796876963

In [122]:
sce=0
for i in range(0, len(df)):
    sce=( y[i] - (Bo+Ao*x[i]) )**2 + sce
S2= sce/(len(df)-2)
S2

0.043679437215751786

In [123]:
tc=b1/sqrt(S2/(l-z**2 /len(df)))
tc

3.4688506448829663

In [119]:
ta=2.07387

In [125]:
scr=Ao*(l-(h/len(df)))
scr

30.515710669772606

In [128]:
cmr=scr/1
cme=sce/len(df)

In [129]:
Fc= cmr/cme
Fc

762.1404557200066

In [130]:
F95=4.3